In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df3_WS3T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_AF_segment3_workflowS3_has_duration.csv",engine='python')

In [3]:
df3_WS3T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,time_used_str,time_used_second
0,1076747|438001|677047,Assessment Outcome - Successful,Assessment Outcome - Successful,15/1/2019 16:46:30,0 days 0 hours 0 minutes 13 seconds,13.0


In [4]:
df3_WS3T['WRKFLW_TSK_MAP_GROUPED'].nunique()

15

In [5]:
df3_WS3T['WRKFLW_TSK_MAP_DETAILED'].nunique()

16

In [6]:
wd_count = df3_WS3T.groupby('WRKFLW_TSK_MAP_DETAILED').WRKFLW_TSK_MAP_DETAILED.count()
wd_dict = dict()
for i, v in wd_count.items():
    wd_dict[i] = v
sorted_wd_dict = sorted(wd_dict.items(), key = lambda x: x[1], reverse = True)
print(sorted_wd_dict)

[('Assessment Outcome - Successful', 245), ('Create Offer', 241), ('Release Offer', 240), ('OC - Created - Evidence of course completion', 22), ('Assessment Decision - Qualified', 8), ('OC - Created - English language requirements', 6), ('AR - Satisfied - English language proficiency', 2), ('Release Assessment Hold', 2), ('AR - Satisfied - Curriculum Vitae', 1), ('AR - Satisfied - English language requirements', 1), ('AR - Satisfied - Further information (1)', 1), ('Assessment Outcome - Higher Preference Successful', 1), ('OC - Created - Further information (1)', 1), ('OC - Satisfied - English language requirements', 1), ('Put Assessment On-Hold', 1), ('Unsuccessful', 1)]


In [7]:
list = df3_WS3T.WRKFLW_TSK_MAP_DETAILED.unique()
avg_time_dict = dict()
for key in list :
    df_slides = df3_WS3T[df3_WS3T['WRKFLW_TSK_MAP_DETAILED'] == key]
    sum = df_slides['time_used_second'].sum()
    c = len(df_slides)
    avg = sum//c
    avg_time_dict[key] = avg

sorted_avg_time_dict = sorted(avg_time_dict.items(),key = lambda x: x[1], reverse = True)
print(sorted_avg_time_dict)

[('Put Assessment On-Hold', 324858.0), ('OC - Satisfied - English language requirements', 278803.0), ('Assessment Decision - Qualified', 117937.0), ('OC - Created - English language requirements', 88137.0), ('Create Offer', 31693.0), ('OC - Created - Further information (1)', 6158.0), ('AR - Satisfied - English language proficiency', 260.0), ('Release Assessment Hold', 158.0), ('OC - Created - Evidence of course completion', 104.0), ('AR - Satisfied - Further information (1)', 39.0), ('Assessment Outcome - Successful', 15.0), ('AR - Satisfied - English language requirements', 13.0), ('Assessment Outcome - Higher Preference Successful', 8.0), ('AR - Satisfied - Curriculum Vitae', 6.0), ('Release Offer', 0.0), ('Unsuccessful', 0.0)]


In [8]:
numOfWorkflowProcesses = df3_WS3T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [9]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [10]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [11]:
list_avgtime_per_key =[]
list_sumtime_row = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df3_WS3T[df3_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))
    list_sumtime_row.append(list_sumtime_person)
list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [12]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df3_WS3T[df3_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [13]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df3_WS3T[df3_WS3T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED']=='Release Offer']
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [14]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df3_WS3T[df3_WS3T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [15]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df3_WS3T[df3_WS3T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_DETAILED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_DETAILED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: step_sum_time[key] // dict_list_step.get(key, 0)
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step[:6])

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)

In [16]:
s3_t3=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s3_t3['Num of Occurrences per step(descending)'] = list_step_count_per_key
s3_t3['Average time spends per step(descending)'] = list_step_ave_time_per_key
s3_t3['average time for workflow segment(seconds)'] = list_avgtime_per_key

s3_t3['sum time for workflow segment(seconds)'] = list_sumtime_row
s3_t3.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,Num of Occurrences per step(descending),Average time spends per step(descending),average time for workflow segment(seconds),sum time for workflow segment(seconds)
0,2,1,0 days 0 hours 0 minutes 16 seconds,0.0 days 0.0 hours 0.0 minutes 16.0 seconds,0.0%,"[(Assessment Outcome - Successful, 1), (Create...","[(Assessment Outcome - Successful, 16.0), (Cre...",16,16.0
1,3,203,0 days 2 hours 58 minutes 14 seconds,0.0 days 0.0 hours 2.0 minutes 36.0 seconds,100.0%,"[(Assessment Outcome - Successful, 203), (Rele...","[(Create Offer, 10731.0), (OC - Created - Evid...",10694,2171025.0
2,4,32,1 days 9 hours 14 minutes 46 seconds,0.0 days 0.0 hours 3.0 minutes 0.5 seconds,96.88%,"[(Assessment Outcome - Successful, 33), (Creat...","[(Create Offer, 119373.0), (OC - Created - Fur...",119686,3829980.0
3,5,3,4 days 0 hours 13 minutes 16 seconds,0.0 days 0.0 hours 2.0 minutes 31.0 seconds,100.0%,"[(Assessment Outcome - Successful, 5), (Create...",[(OC - Created - English language requirements...,346396,1039189.0
4,6,1,13 days 5 hours 41 minutes 52 seconds,13.0 days 5.0 hours 41.0 minutes 52.0 seconds,100.0%,"[(Assessment Outcome - Successful, 1), (Create...","[(Create Offer, 1143159.0), (AR - Satisfied - ...",1143712,1143712.0
5,7,2,8 days 22 hours 22 minutes 34 seconds,8.0 days 22.0 hours 22.0 minutes 34.5 seconds,100.0%,"[(Assessment Outcome - Successful, 2), (Create...","[(Put Assessment On-Hold, 324858.0), (OC - Sat...",771754,1543509.0


In [17]:
s3_t3.to_csv('engineering_AF_segment3_workflowS3_table.csv', index=False)



